In [8]:
from utils import engine
import pandas as pd
from sqlalchemy.exc import ResourceClosedError

In [2]:
end_date = '20181231'
report_date = '20190430'


In [6]:
def execute(sql):
    try:
        df = pd.read_sql(sql, engine)
        return df
    except ResourceClosedError:
        print("No data return!")

In [10]:
## 行业PE
execute('''drop table if exists pe_industry''')
execute(f'''
create table pe_industry as
select trade_date,industry, round(avg(pe_ttm),2) pe_ttm_ind from stock_basic sb
join daily_basic d on sb.ts_code = d.ts_code
where d.trade_date='{report_date}'
group by industry
''')

No data return!
No data return!


C:\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1681, 'Specifying number of digits for floating point data types is deprecated and will be removed in a future release.')
  result = self._query(query)


In [12]:
## 个股PE
execute('''drop table if exists pe_history''')
execute(f'''
create table pe_history as
select '{report_date}' report_date, sb.ts_code, sb.name, round(avg(pe),2) pe_his, round(avg(pe_ttm),2) pe_ttm_his 
from stock_basic sb
join daily_basic db on sb.ts_code=db.ts_code
where trade_date < '{report_date}'
and trade_date > replace(date_sub('{report_date}' , INTERVAL 360 DAY),'-','')
and trade_date > replace(date_add(sb.list_date , INTERVAL 60 DAY),'-','')
group by sb.ts_code
''')

No data return!
No data return!


C:\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1681, 'Specifying number of digits for floating point data types is deprecated and will be removed in a future release.')
  result = self._query(query)


In [16]:
## 分析
execute('''drop table if exists tmp_results''')
execute(f'''
create table tmp_results as
SELECT A.ts_code, sb.industry, sb.name, 
	   dt_roe_ttm, 
       avg_roe, 
       pe, pe_ttm,
       pe_ttm_ind,
       pe_his,
       pe_ttm_his,
       netprofit_yoy_ttm,
       dt_netprofit_yoy_ttm,
       grossprofit_margin_ttm,
       netprofit_margin_ttm
FROM (
	SELECT *, 
        row_number() over (partition by ts_code order by end_date desc) rn
    FROM tmp_ttm
    where end_date < '{report_date}'
    ) A
JOIN (
		SELECT name, round(AVG(dt_roe_ttm),2) AS avg_roe
		FROM tmp_ttm
		where end_date > replace(date_sub('{report_date}' , INTERVAL 540 DAY),'-','')
        and end_date < '{report_date}'
		GROUP BY name
	) B ON A.name = B.name
JOIN daily_basic C ON A.ts_code = C.ts_code
join stock_basic sb on A.ts_code = sb.ts_code
join pe_industry pi on sb.industry = pi.industry
join pe_history ph on A.ts_code = ph.ts_code
WHERE rn = 1
    and C.trade_date = '{report_date}'
    and dt_roe_ttm > 20
	AND avg_roe > 20
	AND pe < 20
	AND pe_ttm < 20
ORDER BY dt_roe_ttm DESC
''')


No data return!
No data return!


C:\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1681, 'Specifying number of digits for floating point data types is deprecated and will be removed in a future release.')
  result = self._query(query)
